<a href="https://colab.research.google.com/github/andrew623849/AI-/blob/master/RNN%E5%BD%B1%E8%A9%95%E8%BE%A8%E8%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import urllib.request
import os
import tarfile
url="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
filepath="aclImdb_v1.tar.gz"
if not os.path.isfile(filepath):
  result=urllib.request.urlretrieve(url,filepath)
  print('downloaded:',result)

In [0]:
if not os.path.exists("aclImdb"):
    tfile=tarfile.open("aclImdb_v1.tar.gz",'r:gz')
    result=tfile.extractall('')

In [0]:
import re
def rm_tag(text):
    re_tag=re.compile(r'<[^>]+>')
    return re_tag.sub('',text)

In [0]:
import os 
def read_files(filetype):
    path="aclImdb/"
    file_list=[]
    
    positive_path=path+filetype+"/pos/"
    for f in os.listdir(positive_path):
        file_list+=[positive_path+f]
        
    negative_path=path+filetype+"/neg/"
    for f in os.listdir(negative_path):
        file_list+=[negative_path+f]
        
    print('read',filetype,'file:',len(file_list))
    
    all_labels=([1]*12500+[0]*12500)
    
    all_texts=[]
    for fi in file_list:
        with open(fi,encoding='utf8') as file_input:
            all_texts+=[rm_tag(" ".join(file_input.readlines()))]
        
    return all_labels,all_texts

In [14]:
y_train,train_text=read_files("train")
y_test,test_text=read_files("test")
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
token=Tokenizer(num_words=3800)
token.fit_on_texts(train_text)
x_train_seq=token.texts_to_sequences(train_text)
x_test_seq=token.texts_to_sequences(test_text)
x_train=sequence.pad_sequences(x_train_seq, maxlen=380)
x_test=sequence.pad_sequences(x_test_seq, maxlen=380)

read train file: 25000
read test file: 25000


In [15]:
from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation,Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN

model=Sequential()
model.add(Embedding(output_dim=32,input_dim=3800,input_length=380))
model.add(Dropout(0.2))
model.add(SimpleRNN(units=16))
model.add(Dense(units=256,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(units=1,activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 380, 32)           121600    
_________________________________________________________________
dropout_4 (Dropout)          (None, 380, 32)           0         
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 16)                784       
_________________________________________________________________
dense_3 (Dense)              (None, 256)               4352      
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 126,993
Trainable params: 126,993
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
train_history=model.fit(x_train,y_train,batch_size=100,epochs=10,verbose=2,validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 - 25s - loss: 0.5030 - acc: 0.7469 - val_loss: 0.3853 - val_acc: 0.8462
Epoch 2/10
 - 24s - loss: 0.3087 - acc: 0.8779 - val_loss: 0.6593 - val_acc: 0.7324
Epoch 3/10
 - 24s - loss: 0.2579 - acc: 0.8993 - val_loss: 0.3847 - val_acc: 0.8312
Epoch 4/10
 - 24s - loss: 0.2279 - acc: 0.9148 - val_loss: 0.7710 - val_acc: 0.7378
Epoch 5/10
 - 24s - loss: 0.2130 - acc: 0.9170 - val_loss: 0.4276 - val_acc: 0.8324
Epoch 6/10
 - 24s - loss: 0.1977 - acc: 0.9248 - val_loss: 0.4809 - val_acc: 0.8316
Epoch 7/10
 - 24s - loss: 0.1718 - acc: 0.9354 - val_loss: 0.4719 - val_acc: 0.8278
Epoch 8/10
 - 24s - loss: 0.1374 - acc: 0.9499 - val_loss: 0.6590 - val_acc: 0.7770
Epoch 9/10
 - 24s - loss: 0.1271 - acc: 0.9538 - val_loss: 0.6540 - val_acc: 0.7812
Epoch 10/10
 - 24s - loss: 0.1249 - acc: 0.9539 - val_loss: 1.1400 - val_acc: 0.7086


In [17]:
scores=model.evaluate(x_test,y_test,verbose=1)
scores[1]

25000/25000 [==============================] - 17s 694us/step


0.82204